In [2]:
%pip install tensorflow

  Using cached tensorflow-2.17.0-cp312-cp312-win_amd64.whl.metadata (3.2 kB)
  Using cached tensorflow_intel-2.17.0-cp312-cp312-win_amd64.whl.metadata (5.0 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached h5py-3.11.0-cp312-cp312-win_amd64.whl.metadata (2.5 kB)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached ml_dtypes-0.4.0-cp312-cp312-win_amd64.whl.metadata (20 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached protobuf-4.25.4-cp310-abi3-win_amd64.whl.metadata (541 bytes)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached wrapt-1.16.0-cp312-cp312-win_amd64.whl.metadata (

In [3]:
import tensorflow as tf

In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory("")

# here 80% of the data is splitted for training
train_df = tf.keras.preprocessing.image_dataset_from_directory(
            "",
            validation_split = 0.2,
            subset='training',
            seed=123
            )
# here 20% data is split for validation
validation_df = tf.keras.preprocessing.image_dataset_from_directory(
            "",
            validation_split=0.2,
            subset = "validation",
            seed=123
            )

In [ ]:
import tensorflow_datasets as tfds

batch_size = 64

dataset_name =dataset
class_names = dataset.class_names
print(class_names)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#num_images = 10

# Here I extracts a batch of images along with their labels
"""image_batch, label_batch = next(iter(train_df))

fig, axes = plt.subplots(2, 5, figsize=(12, 6))
for i, ax in enumerate(axes.flat):
  if i < num_images:
    ax.imshow(image_batch[i].numpy().astype("uint8"))
    ax.set_title(f"{train_df.class_names[label_batch[i].numpy()]}")
    ax.axis('off')
plt.tight_layout()
plt.show()"""


plt.figure(figsize=(10,10))
for images, labels in dataset.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i+1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")
plt.show()

In [ ]:
import keras
from keras.models import Sequential
from tensorflow.keras import layers
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D

Rescale and Resize

In [ ]:
#normalization_layer = tf.keras.layers.Rescaling(1./255)

#train_df = train_df.map(lambda x, y: (normalization_layer(x), y))
#validation_df = validation_df.map(lambda x, y: (normalization_layer(x), y))

image_size = 180

#resizing images to a consistent shape and rescaling the pixel values
resize_rescale = keras.Sequential(
    [
        layers.Resizing(image_size, image_size),
        layers.Rescaling(1./255)
    ]
 )

Data Augmentation

In [ ]:
#for image, label in .take(1):
#  print(f"Image shape : {image.shape}")
data_augmentation = tf.keras.Sequential(
   [
     layers.RandomFlip("horizontal_and_vertical"),
     layers.RandomRotation(0.2),
     layers.RandomZoom(0.1)  ]  )

In [ ]:
image, label = next(iter(dataset))

result = resize_rescale(image[0])
_ = plt.imshow(result)

In [ ]:
train_df = train_df.map(lambda x, y: (resize_rescale(x), y) )
validation_df = validation_df.map(lambda x, y: (resize_rescale(x), y))

In [ ]:
model = Sequential()

model.add(layers.Input(shape=(180, 180, 3)))

model.add(data_augmentation)

model.add(Conv2D(16, 3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, 3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, 3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(23, activation='softmax'))

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(train_df, validation_data=, epochs=30, verbose=1)